In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [37]:
youtube_df = pd.read_excel('kat2C81.xlsx')
youtube_df.columns = ['comment','label']
youtube_df.head()
youtube_df = youtube_df.dropna()
youtube_df['label'] = youtube_df['label'].astype(int)

youtube_df.head()

,comment,label
0,볼스카야에 로봇 퍼시픽림 베낀듯,0
1,도란 도란 관전모드,0
2,이거 아누비스 피라미드에 맥크리와 트레이서 그리고 트레이서와 디바 알맹이가 신문에 ...,1
3,아 이미했구나,0
4,땅밑으로들어갔다고나오면됬어요,0


In [38]:
movie_df = pd.read_csv('ratings_train.txt',encoding='utf8',sep='\t',engine='python')
movie_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [39]:
movie_df = movie_df.iloc[:,[1,2]]
movie_df.columns = ['comment','label']
movie_df = movie_df.dropna()
movie_df.head()

,comment,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [40]:
print(youtube_df.shape)
print(movie_df.shape)

(15133, 2)
(149995, 2)


In [41]:
X1 = youtube_df['comment']
y1 = youtube_df['label']

X2 = movie_df['comment']
y2 = movie_df['label']

In [42]:
from sklearn.model_selection import train_test_split
X1_train,X1_test,y1_train,y1_test = train_test_split(X1,y1,test_size=0.3)

In [43]:
print(X1_train.shape, X2.shape)

(10593,) (149995,)


In [44]:
X_train = X1_train.append(X2)
y_train = y1_train.append(y2)

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [45]:
print(len(X_train), len(y_train))

160588 160588


In [46]:
X_test = X1_test
y_test = y1_test

del X1_test, y1_test

In [47]:
y_train.value_counts()

0    80339
1    80249
Name: label, dtype: int64

In [48]:
import re 
from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    # 형태소 단위로 토큰화 작업
    tokens = okt.morphs(text)
    return tokens

In [49]:
X_train = X_train.apply(lambda x:re.sub(r'[^ ㄱ-ㅣ가-힣]+'," ",x))

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 토큰 생성기는 정의한 함수, 토큰의 단어 크기는 1~2개 단어, 토큰의 출현 빈도가 최소 3번 이상이고 최대 90% 이하인 것들 사용
tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(X_train)
tfidf_train = tfidf.transform(X_train)

In [51]:
from sklearn.linear_model import LogisticRegression
SA_lr = LogisticRegression(random_state=0)
SA_lr.fit(tfidf_train,y_train)

LogisticRegression(random_state=0)

In [52]:
X_test = X_test.apply(lambda x:re.sub(r'[^ ㄱ-ㅣ가-힣]+'," ",x))

In [53]:
tfidf_test = tfidf.transform(X_test)
SA_lr.score(tfidf_test,y_test)

0.7231277533039647

In [55]:
for c in [0.5,1,2,3,3.5,4,4.5]:
    SA_lr = LogisticRegression(random_state=0,C=c)
    SA_lr.fit(tfidf_train,y_train)
    print(c,' : ',SA_lr.score(tfidf_test,y_test))

0.5  :  0.7244493392070485
1  :  0.7231277533039647
2  :  0.7176211453744493
3  :  0.7154185022026431
3.5  :  0.711453744493392
4  :  0.7129955947136564
4.5  :  0.710352422907489
